# Settings

In [1]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep
import sys
# sys.path.append('..' + sep_local + '..' + sep_local +'..' + sep_local + '..' + sep_local + '..'+ sep_local + '..') # For Windows import
os.chdir('..' + sep_local + '..' + sep_local +'..' + sep_local + '..' + sep_local + '..'+ sep_local + '..') # For Linux import
print(sep_local)
print(os.getcwd())

env: TF_KERAS=1
/
/home/azeghost/git/Generative_Models


In [2]:
import tensorflow as tf
print(tf.__version__)

2.2.0


# Dataset loading

In [3]:
dataset_name='atari_pacman'

In [4]:
from training.generators.from_lmdb.lmdb_image_generator import get_generators

In [5]:
# !mkdir /home/azeghost/datasets/

In [6]:
# !cd /home/azeghost/datasets/

In [7]:
# !wget https://omnomnom.vision.rwth-aachen.de/data/atari_v1_release/mspacman.tar.gz -P /home/azeghost/datasets/ --no-check-certificate
    

In [8]:
# !mkdir /home/azeghost/datasets/.mspacman

In [9]:
# !tar -xf /home/azeghost/datasets/mspacman.tar.gz -C /home/azeghost/datasets/.mspacman

In [10]:
# images_dir = '/home/azeghost/datasets/.mspacman/atari_v1/screens/mspacman' #Linux
# #images_dir = 'C:\\projects\\pokemon\DS06\\'
# validation_percentage = 30
# # valid_format = 'png'

In [11]:
# from training.generators.file_image_generator import create_image_lists, get_generators

In [12]:
# imgs_list = create_image_lists(
#     image_dir=images_dir, 
#     validation_pct=validation_percentage, 
#     valid_imgae_formats=valid_format,
#     verbose=0,
#     sequenced = True
# )

In [13]:
# sorted(imgs_list.keys(), key=int)

In [14]:
# imgs_list['1'].keys()

In [15]:
# len(imgs_list['1']['training'])

In [16]:
# sorted(imgs_list['1']['training'], key=lambda x: int(x.split('.')[0]))[:10]

In [17]:
# sorted(imgs_list['1']['validation'], key=lambda x: int(x.split('.')[0]))[-1]

In [18]:
# scale=3
image_size=(53, 70, 3)

batch_size = 32
EPIS_LEN = 10
EPIS_SHIFT = 5

inputs_shape = image_size
latents_dim = 30
intermediate_dim = 30
lmdb_dir = '/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/Pacman_LMDB'

In [19]:
from transformation.lmdb_transformer import LmdbTransformer

In [20]:
train_generator, val_generator = get_generators(
        lmdb_dir=lmdb_dir,
        batch_size=batch_size,
        class_mode='episode', 
        episode_len=EPIS_LEN, 
        episode_shift=EPIS_SHIFT
)

  DEBUG    | Looking for images in '_training'
  DEBUG    | Looking for images in '_training'
  WARNING  | No files found
  WARNING  | No files found
/home/azeghost/git/Generative_Models/transformation/file_image_generator.py:51: UserWarning: No files found
  warnings.warn(msg)
  DEBUG    | Looking for images in '_validation'
  DEBUG    | Looking for images in '_validation'
  WARNING  | No files found
  WARNING  | No files found






Initializing Iterator training Number of images 1092505
Initializing Iterator validation Number of images 468474


In [29]:
training_generator =train_generator
testing_generator = val_generator

In [49]:
for data in train_generator:
    print(data['xt0'].shape)
    print(data['xt1'].shape)
    break

(32, 10, 53, 70, 3)
(32, 10, 53, 70, 3)


In [ ]:
store 70 batches training + 3 val to lmdb

In [ ]:
#redundant

In [31]:
import tensorflow as tf

In [32]:
# train_ds = tf.data.Dataset.from_generator(
#     lambda: training_generator, 
#     output_types=(tf.float32, tf.float32) ,
#     output_shapes=(tf.TensorShape((batch_size, EPIS_LEN, ) + image_size), 
#                    tf.TensorShape((batch_size, EPIS_LEN, ) + image_size)
#                   )
# )

# test_ds = tf.data.Dataset.from_generator(
#     lambda: testing_generator,     
#     output_types=(tf.float32, tf.float32) ,
#     output_shapes=(tf.TensorShape((batch_size, EPIS_LEN, ) + image_size), 
#                    tf.TensorShape((batch_size, EPIS_LEN, ) + image_size)
#                   )
# )

In [33]:
training_generator.next()[0].shape

AttributeError: 'FlatMapDataset' object has no attribute 'next'

In [ ]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break

In [ ]:
_instance_scale = 1.0

In [26]:
import numpy as np
from collections.abc import Iterable
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [27]:
inputs_shape

(53, 70, 3)

In [35]:
next(training_generator)[0].shape

TypeError: 'FlatMapDataset' object is not an iterator

# Model's Layers definition

In [36]:
tdDense = lambda **kwds: tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(**kwds))

In [37]:
enc_lays = [tdDense(units=intermediate_dim//2, activation='relu'),
            tdDense(units=intermediate_dim//2, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latents_dim*EPIS_LEN)]

dec_lays = [tf.keras.layers.Dense(units=latents_dim*EPIS_LEN, activation='relu'),
            tf.keras.layers.Reshape(inputs_shape),
            tdDense(units=intermediate_dim, activation='relu'),
            tdDense(units=_outputs_shape),
            tf.keras.layers.Reshape(inputs_shape)
            ]

# Model definition

In [38]:
model_name = dataset_name+'AE_Dense_reconst_ell'
#windows
#experiments_dir='..' + sep_local + '..' + sep_local +'..' + sep_local + '..' + sep_local + '..'+sep_local+'experiments'+sep_local + model_name

#linux 
experiments_dir=os.getcwd()+ sep_local  +'experiments'+sep_local + model_name

In [41]:
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 842 kB/s eta 0:00:01


In [42]:
from training.autoencoding_basic.transformative.AE import autoencoder as AE

ModuleNotFoundError: No module named 'tensorflow_probability'

In [ ]:
# inputs_shape=image_size

In [ ]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim*EPIS_LEN,
        'layers': enc_lays
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latents_dim*EPIS_LEN,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [ ]:
from os.path import abspath
from utils.data_and_files.file_utils import create_if_not_exist
_restore = os.path.join(experiments_dir, 'var_save_dir')
create_if_not_exist(_restore)
absolute = abspath(_restore)
print("Restore_dir",absolute)
absolute = abspath(experiments_dir)
print("Recording_dir",absolute)
print("Current working dir",os.getcwd())

In [ ]:
#to restore trained model, set filepath=_restore

In [ ]:
ae = AE( 
    name=model_name,
    latents_dim=latents_dim,
    batch_size=batch_size,
    episode_len= EPIS_LEN,
    variables_params=variables_params, 
    filepath=None
    )

In [ ]:
# ae.compile(metrics=None)
ae.compile()

# Callbacks

In [ ]:
import logging
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)

from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [ ]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=12, 
    verbose=1, 
    restore_best_weights=False
)

In [ ]:
ms = ModelSaver(filepath=_restore)

In [ ]:
csv_dir = os.path.join(experiments_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, model_name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
absolute = abspath(csv_dir)
print("Csv_dir",absolute)

In [ ]:
image_gen_dir = os.path.join(experiments_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)
absolute = abspath(image_gen_dir)
print("Image_gen_dir",absolute)

In [ ]:
sg = SampleGeneration(latents_shape=latents_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

# Model Training

In [ ]:
ae.fit(
    x=train_ds,
    input_kw=None,
    steps_per_epoch=10,
    epochs=int(1e6), 
    verbose=2,
    callbacks=[ es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True,
    validation_data=test_ds,
    validation_steps=10
)

# Model Evaluation

## inception_score

In [ ]:
from evaluation.generativity_metrics.inception_metrics import inception_score

In [ ]:
is_mean, is_sigma = inception_score(ae, tolerance_threshold=1e-6, max_iteration=200)
print(f'inception_score mean: {is_mean}, sigma: {is_sigma}')

## Frechet_inception_distance

In [ ]:
from evaluation.generativity_metrics.inception_metrics import frechet_inception_distance

In [ ]:
fis_score = frechet_inception_distance(ae, training_generator, tolerance_threshold=1e-6, max_iteration=10, batch_size=32)
print(f'frechet inception distance: {fis_score}')

## perceptual_path_length_score

In [ ]:
from evaluation.generativity_metrics.perceptual_path_length import perceptual_path_length_score

In [ ]:
ppl_mean_score = perceptual_path_length_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200, batch_size=32)
print(f'perceptual path length score: {ppl_mean_score}')

## precision score

In [ ]:
from evaluation.generativity_metrics.precision_recall import precision_score

In [ ]:
_precision_score = precision_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200)
print(f'precision score: {_precision_score}')

## recall score

In [ ]:
from evaluation.generativity_metrics.precision_recall import recall_score

In [ ]:
_recall_score = recall_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200)
print(f'recall score: {_recall_score}')

# Image Generation

## image reconstruction

### Training dataset

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from training.generators.image_generation_testing import reconstruct_from_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, testing_generator, save_dir)

## with Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_like_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, testing_generator, save_dir)

### Complete Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_randomly

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'random_synthetic_dir')
create_if_not_exist(save_dir)

generate_images_randomly(ae, save_dir)

In [ ]:
from training.generators.image_generation_testing import interpolate_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'interpolate_dir')
create_if_not_exist(save_dir)

interpolate_a_batch(ae, testing_generator, save_dir)

### Stacked inputs outputs and predictions 

In [ ]:
from training.generators.image_generation_testing import predict_from_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'predictions')
create_if_not_exist(save_dir)

predict_from_a_batch(ae, testing_generator, save_dir)